In [1]:
import os, re, copy, json, pprint, argparse, warnings, xmltojson, shutil
import pandas as pd
from pathlib import Path
from collections import Counter
warnings.filterwarnings('ignore')

In [26]:
with open("financial_anomalies_cited_network_raw", "r") as fp:
    academic_networks = json.load(fp)
counter = 0
for item in academic_networks:
    if not isinstance(item.get('main_title'), type(None)) | isinstance(item.get('cited_titles'), type(None)):
        counter +=  1
        print(counter, item.get('main_title'), len(item.get('cited_titles')))

1 Can financial statement analysis beat consensus analysts' recommendations? 30
2 Financial Constraints and Stock Returns 29
3 The Capital Asset Pricing Model: Some Empirical Tests 20
4 Stock Returns and Volatility: Pricing the Short-Run and Long-Run Components of Market Risk 71
5 Do Heterogeneous Beliefs Matter for Asset Pricing? 66
6 The relation between corporate financing activities, analysts' forecasts and stock returns $ 28
7 An Alternative Three-Factor Model 37
8 Financing intangible capital R 40
9 NBER WORKING PAPER SERIES BAD BETA, GOOD BETA 60
10 Labor Leverage, Firms' Heterogeneous Sensitivities to the Business Cycle, and the Cross-Section of Expected Returns 34
11 NBER WORKING PAPER SERIES SHORT SALE CONSTRAINTS AND STOCK RETURNS 19
12 Predicting Future Earnings Changes Using Machine Learning and Detailed Financial Data 69
13 Writing matters 17
14 Predicting Stock Returns Using Industry-Relative Firm Characteristics 1 (Please do not quote without permission) 26
15 Do Invest

In [ ]:
!clear
!git add .
!git status
!git commit -m "citations graph"
!git push
!clear 
